In [1]:
from yahoo_fin.stock_info import get_data

# Test

In [6]:
cocoa_weekly = get_data("cc=f", start_date='01/01/2024', end_date='06/09/2024', index_as_date=True, interval='1wk')
cocoa_weekly

,open,high,low,close,adjclose,volume,ticker
2024-01-01,4209.0,4287.0,4194.0,4204.0,4204.0,84667,CC=F
2024-01-08,4194.0,4358.0,4034.0,4323.0,4323.0,159349,CC=F
2024-01-15,4290.0,4607.0,4285.0,4583.0,4583.0,162379,CC=F
2024-01-22,4589.0,4840.0,4529.0,4672.0,4672.0,184917,CC=F
2024-01-29,4679.0,5032.0,4632.0,5009.0,5009.0,163327,CC=F
2024-02-05,5035.0,6030.0,5010.0,5888.0,5888.0,189221,CC=F
2024-02-12,5868.0,6039.0,5689.0,5706.0,5706.0,54109,CC=F
2024-02-19,5947.0,6750.0,5689.0,6562.0,6562.0,21018,CC=F
2024-02-26,6765.0,6929.0,6362.0,6826.0,6826.0,381,CC=F
2024-03-04,7057.0,7170.0,6928.0,6997.0,6997.0,6,CC=F


# Commodities of Interest (Ticker Index)

1. **Corn:** ZC=F
2. **Wheat:** KE=F   
3. **Soybeans:** ZS=F 
4. **Rough Rice:** ZR=F 
5. **Oats:** ZO=F 
6. **Coffee:** KC=F 
7. **Cocoa:** CC=F 
8. **Cotton:** CT=F 
9. **Sugar:** SB=F 
10. **Orange Juice:** OJ=F 

# Data

In [50]:
# commodity_index = ['corn', 'wheat', 'soybeans', 'rough rice', 'oats', 'coffee', 'cocoa', 'cotton', 'sugar', 'orange juice']
commodities = ['zc=f', 'ke=f', 'zs=f', 'zr=f', 'zo=f', 'kc=f', 'cc=f', 'ct=f', 'sb=f', 'oj=f']
historical_data = {}

In [51]:
# No retreival parameters set
for commodity in commodities:
    historical_data[commodity] = get_data(commodity)

In [52]:
# Number of records per commodity ticker
for commodity in commodities:
    print(commodity, len(historical_data[commodity]))

zc=f 6081
ke=f 6033
zs=f 6037
zr=f 6221
zo=f 6221
kc=f 6221
cc=f 6221
ct=f 6221
sb=f 6179
oj=f 697


In [53]:
# Sample of collected data
historical_data['kc=f']

,open,high,low,close,adjclose,volume,ticker
2000-01-03,124.000000,124.000000,116.099998,116.500000,116.500000,6640.0,KC=F
2000-01-04,116.500000,120.500000,115.750000,116.250000,116.250000,5492.0,KC=F
2000-01-05,115.000000,121.000000,115.000000,118.599998,118.599998,6165.0,KC=F
2000-01-06,119.000000,121.400002,116.500000,116.849998,116.849998,5094.0,KC=F
2000-01-07,117.750000,117.750000,113.800003,114.150002,114.150002,6855.0,KC=F
...,...,...,...,...,...,...,...
2024-06-05,230.250000,233.250000,226.350006,231.949997,231.949997,29236.0,KC=F
2024-06-06,232.000000,239.800003,231.800003,234.199997,234.199997,30868.0,KC=F
2024-06-07,233.449997,234.100006,223.600006,224.800003,224.800003,49022.0,KC=F
2024-06-10,225.000000,226.449997,219.050003,221.399994,221.399994,49022.0,KC=F


# Commodities of Interest (Start Date)

Used to dictate start date for weather API call...

1. **Corn:** July 17, 2000
2. **Wheat:** Sept. 21, 2000   
3. **Soybeans:** Sept. 15, 2000
4. **Rough Rice:** Jan. 3, 2000 
5. **Oats:** Jan. 3, 2000  
6. **Coffee:** Jan. 3, 2000 
7. **Cocoa:** Jan. 3, 2000  
8. **Cotton:** Jan. 3, 2000  
9. **Sugar:** March 1, 2000
10. **Orange Juice:** Sept. 7, 2021 